In [ ]:
# Get helper functions file
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-04-04 09:11:00--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-04-04 09:11:00 (80.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
# Get helper functions file
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-04-04 09:11:05--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-04-04 09:11:05 (109 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
# Download data from Google Storage (already preformatted)
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip 

unzip_data("101_food_classes_10_percent.zip")

train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

--2023-04-04 09:13:22--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip.1’

101_food_classes_10 100%[===================>]   1.51G  22.1MB/s    in 69s     

2023-04-04 09:14:32 (22.6 MB/s) - ‘101_food_classes_10_percent.zip.1’ saved [1625420029/1625420029]



In [ ]:
# Step 1: Import necessary libraries
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Step 2: Load the dataset
data_dir = "101_food_classes_10_percent"
train_dir = f"{data_dir}/train"
test_dir = f"{data_dir}/test"

In [ ]:
train_dir

'101_food_classes_10_percent/train'

In [ ]:
test_dir

'101_food_classes_10_percent/test'

In [ ]:
# Step 3: Preprocess the images
image_size = (528, 528)  # EfficientNet-B6 input size
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=image_size,
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=image_size,
                                                  batch_size=batch_size,
                                                  class_mode='categorical')


Found 7575 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


In [ ]:
# Step 4: Create the model using pre-trained EfficientNet-B6 feature vector
num_classes = 101

feature_extractor_url = "https://tfhub.dev/tensorflow/efficientnet/b6/feature-vector/1"

feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(528, 528, 3),
                                         trainable=False)  # Freeze the feature extractor for initial training

model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Step 5: Train the model with feature extraction
initial_epochs = 10
history = model.fit(train_generator,
                    epochs=initial_epochs,
                    validation_data=test_generator)


In [ ]:
# Step 6: Fine-tune the model
# Unfreeze the feature extractor layer
feature_extractor_layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

history_fine_tune = model.fit(train_generator,
                              epochs=total_epochs,
                              initial_epoch=history.epoch[-1],
                              validation_data=test_generator)


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

def predict_image(img_path, model):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    # Make predictions
    preds = model.predict(x)
    predicted_class_index = np.argmax(preds[0])
    confidence_score = np.max(preds[0])

    # Get the class labels and predicted class label
    class_labels = list(train_generator.class_indices.keys())
    predicted_class = class_labels[predicted_class_index]

    # Display the image, predicted class, and confidence score
    plt.imshow(img)
    plt.title(f"Predicted: {predicted_class} ({confidence_score:.2f})")
    plt.axis('off')
    plt.show()

    return preds

# Example: predict_image("path/to/image.jpg", model)


In [ ]:
predict_image("path/to/image.jpg", model)


In [ ]:
import os
import random

def predict_random_image(model, test_dir):
    # Select a random food class directory from the test data
    food_class_dirs = [d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))]
    random_food_class = random.choice(food_class_dirs)
    random_food_class_dir = os.path.join(test_dir, random_food_class)

    # Select a random image from the chosen food class directory
    image_files = [f for f in os.listdir(random_food_class_dir) if os.path.isfile(os.path.join(random_food_class_dir, f))]
    random_image_file = random.choice(image_files)
    random_image_path = os.path.join(random_food_class_dir, random_image_file)

    # Load and preprocess the image
    img = image.load_img(random_image_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    # Make predictions
    preds = model.predict(x)
    predicted_class_index = np.argmax(preds[0])
    confidence_score = np.max(preds[0])

    # Get the class labels and predicted class label
    class_labels = list(train_generator.class_indices.keys())
    predicted_class = class_labels[predicted_class_index]

    # Display the image, predicted class, and the actual class
    plt.imshow(img)
    plt.title(f"Predicted: {predicted_class} ({confidence_score:.2f})\nActual: {random_food_class}")
    plt.axis('off')
    plt.show()

    return preds



In [ ]:
predict_random_image(model, test_dir)


In [ ]:
# Step 4: Create the model using pre-trained EfficientNet-B6 feature vector with GlobalAveragePooling2D
num_classes = 101

feature_extractor_url = "https://tfhub.dev/tensorflow/efficientnet/b6/feature-vector/1"

feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(528, 528, 3),
                                         trainable=False)  # Freeze the feature extractor for initial training

model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 5: Train the model with feature extraction
initial_epochs = 10

checkpoint_filepath_feature_extraction = "feature_extraction_checkpoint.h5"
model_checkpoint_callback_feature_extraction = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_feature_extraction,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1)

history = model.fit(train_generator,
                    epochs=initial_epochs,
                    validation_data=test_generator,
                    callbacks=[model_checkpoint_callback_feature_extraction])

# Step 6: Fine-tune the model
# Unfreeze the feature extractor layer
feature_extractor_layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

checkpoint_filepath_fine_tuning = "fine_tuning_checkpoint.h5"
model_checkpoint_callback_fine_tuning = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_fine_tuning,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1)

history_fine_tune = model.fit(train_generator,
                              epochs=total_epochs,
                              initial_epoch=history.epoch[-1],
                              validation_data=test_generator,
                              callbacks=[model_checkpoint_callback_fine_tuning])


In [ ]:
# Step 5: Train the model with feature extraction
initial_epochs = 10

checkpoint_filepath_feature_extraction = "feature_extraction_weights_checkpoint.h5"
model_checkpoint_callback_feature_extraction = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_feature_extraction,
    save_best_only=True,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1)

history = model.fit(train_generator,
                    epochs=initial_epochs,
                    validation_data=test_generator,
                    callbacks=[model_checkpoint_callback_feature_extraction])

# Load the best weights from the feature extraction phase before fine-tuning
model.load_weights(checkpoint_filepath_feature_extraction)

# Step 6: Fine-tune the model
# Unfreeze the feature extractor layer
feature_extractor_layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

checkpoint_filepath_fine_tuning = "fine_tuning_checkpoint.h5"
model_checkpoint_callback_fine_tuning = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_fine_tuning,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1)

history_fine_tune = model.fit(train_generator,
                              epochs=total_epochs,
                              initial_epoch=history.epoch[-1],
                              validation_data=test_generator,
                              callbacks=[model_checkpoint_callback_fine_tuning])
